# Model Performance And Results

**Content**:
1. [Libraries](#libraries)
2. [Gridworld Creation](#gridworld)
3. [Functions](#funct)
4. [Uniform Dataset MLP Performance](#mlpuniform)
5. [Non-Uniform Dataset MLP Performance](#mlpnonuniform)

## 1. Libraries <a id = "libraries"> 

In [1]:
import numpy as np
import random
import sys
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, validation_curve
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer

## 2. Gridworld Creation <a id = "gridworld">

In [2]:
def randPair(s,e):
    return np.random.randint(s,e), np.random.randint(s,e)

class BoardPiece:

    def __init__(self, name, code, pos):
        self.name = name #name of the piece
        self.code = code #an ASCII character to display on the board
        self.pos = pos #2-tuple e.g. (1,4)

class BoardMask:

    def __init__(self, name, mask, code):
        self.name = name
        self.mask = mask
        self.code = code

    def get_positions(self): #returns tuple of arrays
        return np.nonzero(self.mask)

def zip_positions2d(positions): #positions is tuple of two arrays
    x,y = positions
    return list(zip(x,y))

class GridBoard:

    def __init__(self, size=4):
        self.size = size #Board dimensions, e.g. 4 x 4
        self.components = {} #name : board piece
        self.masks = {}

    def addPiece(self, name, code, pos=(0,0)):
        newPiece = BoardPiece(name, code, pos)
        self.components[name] = newPiece

    #basically a set of boundary elements
    def addMask(self, name, mask, code):
        #mask is a 2D-numpy array with 1s where the boundary elements are
        newMask = BoardMask(name, mask, code)
        self.masks[name] = newMask

    def movePiece(self, name, pos):
        move = True
        for _, mask in self.masks.items():
            if pos in zip_positions2d(mask.get_positions()):
                move = False
        if move:
            self.components[name].pos = pos

    def delPiece(self, name):
        del self.components['name']

    def render(self):
        dtype = '<U2'
        displ_board = np.zeros((self.size, self.size), dtype=dtype)
        displ_board[:] = ' '

        for name, piece in self.components.items():
            displ_board[piece.pos] = piece.code

        for name, mask in self.masks.items():
            displ_board[mask.get_positions()] = mask.code

        return displ_board

    def render_np(self):
        num_pieces = len(self.components) + len(self.masks)
        displ_board = np.zeros((num_pieces, self.size, self.size), dtype=np.uint8)
        layer = 0
        for name, piece in self.components.items():
            pos = (layer,) + piece.pos
            displ_board[pos] = 1
            layer += 1

        for name, mask in self.masks.items():
            x,y = self.masks['boundary'].get_positions()
            z = np.repeat(layer,len(x))
            a = (z,x,y)
            displ_board[a] = 1
            layer += 1
        return displ_board

def addTuple(a,b):
    return tuple([sum(x) for x in zip(a,b)])

In [3]:
class Gridworld:

    def __init__(self, size=4, mode='static'):
        if size >= 4:
            self.board = GridBoard(size=size)
        else:
            print("Minimum board size is 4. Initialized to size 4.")
            self.board = GridBoard(size=4)

        #Add pieces, positions will be updated later
        self.board.addPiece('Player','P',(0,0))
        self.board.addPiece('Goal','+',(0,3))
        self.board.addPiece('Pit','-',(0,3))
        self.board.addPiece('Wall','W',(2,3))

        if mode == 'static':
            self.initGridStatic()
        elif mode == 'player':
            self.initGridPlayer()
        else:
            self.initGridRand()

    #Initialize stationary grid, all items are placed deterministically
    def initGridStatic(self):
        #Setup static pieces
        self.board.components['Player'].pos = randPair(0,self.board.size) #Row, Column
        self.board.components['Goal'].pos = (3,0)
        self.board.components['Pit'].pos = (3,1)
        self.board.components['Wall'].pos = (3,2)
        
        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridStatic()


    #Check if board is initialized appropriately (no overlapping pieces)
    #also remove impossible-to-win boards
    def validateBoard(self):
        valid = True

        player = self.board.components['Player']
        goal = self.board.components['Goal']
        wall = self.board.components['Wall']
        pit = self.board.components['Pit']

        all_positions = [piece for name,piece in self.board.components.items()]
        all_positions = [player.pos, goal.pos, wall.pos, pit.pos]
        if len(all_positions) > len(set(all_positions)):
            return False

        corners = [(0,0),(0,self.board.size), (self.board.size,0), (self.board.size,self.board.size)]
        #if player is in corner, can it move? if goal is in corner, is it blocked?
        if player.pos in corners or goal.pos in corners:
            val_move_pl = [self.validateMove('Player', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            val_move_go = [self.validateMove('Goal', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            if 0 not in val_move_pl or 0 not in val_move_go:
                #print(self.display())
                #print("Invalid board. Re-initializing...")
                valid = False

        return valid

    #Initialize player in random location, but keep wall, goal and pit stationary
    def initGridPlayer(self):
        #height x width x depth (number of pieces)
        self.initGridStatic()
        #place player
        self.board.components['Player'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridPlayer()

    #Initialize grid so that goal, pit, wall, player are all randomly placed
    def initGridRand(self):
        #height x width x depth (number of pieces)
        self.board.components['Player'].pos = randPair(0,self.board.size)
        self.board.components['Goal'].pos = randPair(0,self.board.size)
        self.board.components['Pit'].pos = randPair(0,self.board.size)
        self.board.components['Wall'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridRand()

    def validateMove(self, piece, addpos=(0,0)):
        outcome = 0 #0 is valid, 1 invalid, 2 lost game
        pit = self.board.components['Pit'].pos
        wall = self.board.components['Wall'].pos
        new_pos = addTuple(self.board.components[piece].pos, addpos)
        if new_pos == wall:
            outcome = 1 #block move, player can't move to wall
        elif max(new_pos) > (self.board.size-1):    #if outside bounds of board
            outcome = 1
        elif min(new_pos) < 0: #if outside bounds
            outcome = 1
        elif new_pos == pit:
            outcome = 2

        return outcome

    def makeMove(self, action):
        #need to determine what object (if any) is in the new grid spot the player is moving to
        #actions in {u,d,l,r}
        def checkMove(addpos):
            if self.validateMove('Player', addpos) in [0,2]:
                new_pos = addTuple(self.board.components['Player'].pos, addpos)
                self.board.movePiece('Player', new_pos)

        if action == 'u': #up
            checkMove((-1,0))
        elif action == 'd': #down
            checkMove((1,0))
        elif action == 'l': #left
            checkMove((0,-1))
        elif action == 'r': #right
            checkMove((0,1))
        else:
            pass

    def reward(self):
        if (self.board.components['Player'].pos == self.board.components['Pit'].pos):
            return -10
        elif (self.board.components['Player'].pos == self.board.components['Goal'].pos):
            return 10
        else:
            return -1

    def display(self):
        return self.board.render()

In [4]:
action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}

## 3. Functions <a id = 'funct'> 

In [6]:
"""
from2dto1d:
    Args: 
        pos(tuple):2d position of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(integer): 1d position of the objects(Player, Wall, Goal, Pit)in the gridworld
""" 
def from2dto1d(pos):
    if pos == '(0, 0)':
        return 0
    if pos == '(0, 1)':
        return 1
    if pos == '(0, 2)':
        return 2
    if pos == '(0, 3)':
        return 3
    if pos == '(0, 4)':
        return 4
    
    if pos == '(1, 0)':
        return 5
    if pos == '(1, 1)':
        return 6
    if pos == '(1, 2)':
        return 7
    if pos == '(1, 3)':
        return 8
    if pos == '(1, 4)':
        return 9
    
    if pos == '(2, 0)':
        return 10
    if pos == '(2, 1)':
        return 11
    if pos == '(2, 2)':
        return 12
    if pos == '(2, 3)':
        return 13
    if pos == '(2, 4)':
        return 14   
    
    if pos == '(3, 0)':
        return 15
    if pos == '(3, 1)':
        return 16
    if pos == '(3, 2)':
        return 17
    if pos == '(3, 3)':
        return 18
    if pos == '(3, 4)':
        return 19
    
    if pos == '(4, 0)':
        return 20
    if pos == '(4, 1)':
        return 21
    if pos == '(4, 2)':
        return 22
    if pos == '(4, 3)':
        return 23
    if pos == '(4, 4)':
        return 24

In [7]:
def from1dto2d(pos):
    if pos == 0:
        return (0, 0)
    if pos == 1:
        return (0, 1)
    if pos == 2:
        return (0, 2)
    if pos == 3:
        return (0, 3)
    if pos == 4:
        return (0, 4)  
    
    if pos == 5:
        return (1, 0)
    if pos == 6:
        return (1, 1)
    if pos == 7:
        return (1, 2)
    if pos == 8:
        return (1, 3)
    if pos == 9:
        return (1, 4)
    
    if pos == 10:
        return (2, 0)
    if pos == 11:
        return (2, 1)
    if pos == 12:
        return (2, 2)
    if pos == 13:
        return (2, 3)
    if pos == 14:
        return (2, 4)    

    if pos == 15:
        return (3, 0)
    if pos == 16:
        return (3, 1)
    if pos == 17:
        return (3, 2)
    if pos == 18:
        return (3, 3)
    if pos == 19:
        return (3, 4)    
    
    if pos == 20:
        return (4, 0)
    if pos == 21:
        return (4, 1)
    if pos == 22:
        return (4, 2)
    if pos == 23:
        return (4, 3)
    if pos == 24:
        return (4, 4)  

In [8]:
"""
from_num_to_one_hot_encode:
    Args: 
        num(int): number from 0 to 15 representing the state of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(tuple): encoded binary code with the size of 16-bit 
"""
def from_num_to_one_hot_encode(num):
    en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
    if num == 0:
        en0 = 1
    if num == 1:
        en1 = 1
    if num == 2:
        en2 = 1
    if num == 3:
        en3 = 1
    if num == 4:
        en4 = 1
    if num == 5:
        en5 = 1
    if num == 6:
        en6 = 1
    if num == 7:
        en7 = 1
    if num == 8:
        en8 = 1
    if num == 9:
        en9 = 1
    if num == 10:
        en10 = 1
    if num == 11:
        en11 = 1
    if num == 12:
        en12 = 1
    if num == 13:
        en13 = 1
    if num == 14:
        en14 = 1
    if num == 15:
        en15 = 1
    return en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15

In [9]:
def gridworld_test_model(model_b, mode='static', display=True):
    i = 0
    test_game = Gridworld(size=5, mode=mode)
    if display:
        print("Initial State:")
        print(test_game.display())
    status = 1
    j = 0
    while(status == 1): #A
        p_curr = from2dto1d((str(test_game.board.components['Player'].pos)))
        pi = from2dto1d((str(test_game.board.components['Pit'].pos)))
        g = from2dto1d((str(test_game.board.components['Goal'].pos)))
        w = from2dto1d((str(test_game.board.components['Wall'].pos)))

        q_value_next = []

        test_game.makeMove('u')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        
        if p_curr != p:
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('d')
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('d')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        
        if p_curr != p:
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('u')
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('l')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        

        if p_curr != p:
            test_game.makeMove('r')
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('r')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        

        if p_curr != p:
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('l')
        else:
            q_value_next.append(-10)
        
        
        
        action_ = np.argmax(q_value_next)
        action = action_set[action_]
        
        
    
        if display:
            print('Move #: %s; Taking action: %s' % (i, action))
        test_game.makeMove(action)

        j += 1
        if display:
            print(test_game.display())
        reward = test_game.reward()
        if reward != -1:
            if reward > 0:
                status = 2
                if display:
                    print("Game won! Reward: %s" % (reward,))
            else:
                status = 0
                if display:
                    print("Game LOST. Reward: %s" % (reward,))
        i += 1
        if (i > 15):
            if display:
                print("Game lost; too many moves.")
            break
    
    win = True if status == 2 else False
    return win

### Datasets

In [10]:
df_uni = pd.read_csv("DatasetAfterUniform_5x5.csv")
df_uni

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Bellman_Operator
0,0,0,2,1,0,4.099013,-10,2.460884
1,1,2,2,1,0,8.792521,10,10.000000
2,2,3,2,1,0,8.045016,-1,10.000000
3,3,4,2,1,0,5.620292,-1,6.240515
4,4,5,2,1,0,3.845427,-1,3.094377
...,...,...,...,...,...,...,...,...
331195,331195,20,22,23,24,6.237135,-1,3.814733
331196,331196,22,22,23,24,8.608895,10,10.000000
331197,331197,17,22,23,24,5.773876,-1,10.000000
331198,331198,21,22,23,24,5.349704,-1,10.000000


In [64]:
df_non_uni = pd.read_csv("DatasetAfterNonUniform_5x5.csv")
df_non_uni

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Bellman_Operator
0,0,19,4,5,24,4.973315,-1.0,2.524690
1,1,14,4,5,24,8.101170,-1.0,6.024459
2,2,9,4,5,24,7.804955,-1.0,10.000000
3,3,20,12,11,6,1.716758,-1.0,2.809987
4,4,21,12,11,6,4.233319,-1.0,4.670619
...,...,...,...,...,...,...,...,...
13165,13165,14,6,7,18,2.391237,-1.0,2.768155
13166,13166,13,6,7,18,2.209640,-1.0,4.532734
13167,13167,12,6,7,18,3.876474,-1.0,5.948712
13168,13168,16,6,7,18,6.785964,-1.0,6.425222


## 4. Unform Dataset MLP Performance <a id = "mlpuniform"> 

In [13]:
X = df_uni[['Player','Wall','Goal','Pit']]
y = df_uni[['Q_Current']]

In [14]:
X

,Player,Wall,Goal,Pit
0,0,1,2,0
1,2,1,2,0
2,3,1,2,0
3,4,1,2,0
4,5,1,2,0
...,...,...,...,...
331195,20,23,22,24
331196,22,23,22,24
331197,17,23,22,24
331198,21,23,22,24


In [15]:
y

,Q_Current
0,4.099013
1,8.792521
2,8.045016
3,5.620292
4,3.845427
...,...
331195,6.237135
331196,8.608895
331197,5.773876
331198,5.349704


In [16]:
mlp_uniform_q = MLPRegressor(hidden_layer_sizes=(120,100), max_iter=300, activation = 'relu', solver='adam', random_state=123)

In [17]:
mlp_uniform_q.fit(X.values,y.values.ravel())

c:\Users\kuany\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(120, 100), max_iter=300, random_state=123)

In [18]:
gridworld_test_model(mlp_uniform_q, 'random', display = True)

Initial State:
[['-' ' ' ' ' ' ' ' ']
 [' ' 'P' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' '+']
 [' ' ' ' ' ' 'W' ' ']]
Move #: 0; Taking action: d
[['-' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' 'P' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' '+']
 [' ' ' ' ' ' 'W' ' ']]
Move #: 1; Taking action: r
[['-' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'P' ' ' ' ']
 [' ' ' ' ' ' ' ' '+']
 [' ' ' ' ' ' 'W' ' ']]
Move #: 2; Taking action: r
[['-' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 'P' ' ']
 [' ' ' ' ' ' ' ' '+']
 [' ' ' ' ' ' 'W' ' ']]
Move #: 3; Taking action: l
[['-' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'P' ' ' ' ']
 [' ' ' ' ' ' ' ' '+']
 [' ' ' ' ' ' 'W' ' ']]
Move #: 4; Taking action: r
[['-' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 'P' ' ']
 [' ' ' ' ' ' ' ' '+']
 [' ' ' ' ' ' 'W' ' ']]
Move #: 5; Taking action: l
[['-' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'P' ' ' ' ']
 [' ' ' ' ' ' ' ' '+']
 [' ' ' ' ' ' 'W' ' ']]
Move 

False

In [19]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_q, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 847
0.847
0.847
Games played: 1000, # of wins: 864
0.864
1.7109999999999999
Games played: 1000, # of wins: 861
0.861
2.572
Games played: 1000, # of wins: 861
0.861
3.433
Games played: 1000, # of wins: 855
0.855
4.288
Games played: 1000, # of wins: 880
0.88
5.168
Games played: 1000, # of wins: 862
0.862
6.03
Games played: 1000, # of wins: 876
0.876
6.906000000000001
Games played: 1000, # of wins: 830
0.83
7.736000000000001
Games played: 1000, # of wins: 865
0.865
8.601
Win percentage: 86.01%


### Bellman Operator Approximation

In [16]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Bellman_Operator']]

In [18]:
mlp_uniform_bo = MLPRegressor(hidden_layer_sizes=(120,100), max_iter=300, activation = 'relu', solver='adam', random_state=123)

In [19]:
mlp_uniform_bo.fit(X.values,y.values.ravel())

c:\Users\kuany\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(120, 100), max_iter=300, random_state=123)

In [21]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_bo, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 906
0.906
0.906
Games played: 1000, # of wins: 909
0.909
1.815
Games played: 1000, # of wins: 918
0.918
2.733
Games played: 1000, # of wins: 910
0.91
3.6430000000000002
Games played: 1000, # of wins: 909
0.909
4.5520000000000005
Games played: 1000, # of wins: 917
0.917
5.469
Games played: 1000, # of wins: 896
0.896
6.365
Games played: 1000, # of wins: 918
0.918
7.283
Games played: 1000, # of wins: 905
0.905
8.188
Games played: 1000, # of wins: 926
0.926
9.114
Win percentage: 91.14000000000001%


## 5. Non-Unform Dataset MLP Performance <a id = "mlpnonuniform"> 

### Q value approximation

In [32]:
X = df_non_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_non_uni[['Q_Current']]

In [53]:
mlp_non_uniform_q = MLPRegressor(hidden_layer_sizes=(120, 100), max_iter=100, activation = 'relu', solver='adam', random_state=123)

In [54]:
mlp_non_uniform_q.fit(X.values,y.values.ravel())

MLPRegressor(hidden_layer_sizes=(120, 100), max_iter=100, random_state=123)

In [55]:
gridworld_test_model(mlp_non_uniform_q, 'random', display = True)

Initial State:
[[' ' 'W' ' ' ' ' ' ']
 [' ' ' ' ' ' 'P' ' ']
 [' ' ' ' ' ' ' ' ' ']
 ['+' ' ' '-' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Move #: 0; Taking action: d
[[' ' 'W' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 'P' ' ']
 ['+' ' ' '-' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Move #: 1; Taking action: r
[[' ' 'W' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'P']
 ['+' ' ' '-' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Move #: 2; Taking action: l
[[' ' 'W' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 'P' ' ']
 ['+' ' ' '-' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Move #: 3; Taking action: r
[[' ' 'W' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'P']
 ['+' ' ' '-' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Move #: 4; Taking action: l
[[' ' 'W' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' 'P' ' ']
 ['+' ' ' '-' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Move #: 5; Taking action: r
[[' ' 'W' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'P']
 ['+' ' ' '-' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Move 

False

In [56]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_non_uniform_q, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 579
0.579
0.579
Games played: 1000, # of wins: 566
0.566
1.145
Games played: 1000, # of wins: 558
0.558
1.703
Games played: 1000, # of wins: 574
0.574
2.277
Games played: 1000, # of wins: 571
0.571
2.848
Games played: 1000, # of wins: 543
0.543
3.391
Games played: 1000, # of wins: 573
0.573
3.964
Games played: 1000, # of wins: 559
0.559
4.523
Games played: 1000, # of wins: 560
0.56
5.083
Games played: 1000, # of wins: 533
0.533
5.6160000000000005
Win percentage: 56.16000000000001%


### Bellman Operator Approximation

In [57]:
X = df_non_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_non_uni[['Bellman_Operator']]

In [61]:
mlp_non_uniform_bo = MLPRegressor(hidden_layer_sizes=(100,100), max_iter=300, activation = 'relu', solver='adam', random_state=123)

In [62]:
mlp_non_uniform_bo.fit(X.values,y.values.ravel())

MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=300, random_state=123)

In [63]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_non_uniform_bo, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 729
0.729
0.729
Games played: 1000, # of wins: 721
0.721
1.45
Games played: 1000, # of wins: 738
0.738
2.1879999999999997
Games played: 1000, # of wins: 727
0.727
2.9149999999999996
Games played: 1000, # of wins: 757
0.757
3.6719999999999997
Games played: 1000, # of wins: 729
0.729
4.401
Games played: 1000, # of wins: 716
0.716
5.117
Games played: 1000, # of wins: 717
0.717
5.834
Games played: 1000, # of wins: 750
0.75
6.584
Games played: 1000, # of wins: 728
0.728
7.311999999999999
Win percentage: 73.11999999999999%
